# CS 598 PSL Project 3: approach based on Campuswire post [628](https://campuswire.com/c/G06C55090/feed/628)

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

SEED = 4031
np.random.seed(SEED)

In [5]:
# Pull in datasets
train_datasets = []
test_datasets = []
test_ys = []

num_folds = 5

for fold in range(num_folds):
    folder = f"Data/split_{fold+1}/"
    train_datasets.append(pd.read_csv(folder + "train.tsv", sep="\t"))
    test_datasets.append(pd.read_csv(folder + "test.tsv", sep="\t"))
    test_ys.append(pd.read_csv(folder + "test_y.tsv", sep="\t"))

In [6]:
# Custom stopword list
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "their", "they", "his", \
             "her", "she", "he", "a", "an", "and", "is", "was", "are", "were", "him", "himself", "has", "have", "it", "its", \
             "the", "us"]

## Construct vocabulary

In [7]:
# Use all training data to construct vocabulary.

all_train = pd.concat(train_datasets, axis=0, ignore_index=True)
all_train.drop(columns=["id"], inplace=True)
all_train


,sentiment,review
0,1,Naturally in a film who's main themes are of m...
1,0,Afraid of the Dark left me with the impression...
2,0,This has to be one of the biggest misfires eve...
3,0,"This is one of those movies I watched, and won..."
4,0,This movie was dreadful. Biblically very inacc...
...,...,...
124995,0,"I am a student of film, and have been for seve..."
124996,0,It seems like more consideration has gone into...
124997,0,I don't believe they made this film. Completel...
124998,0,This 30 minute documentary Buñuel made in the ...


### Preprocess text

In [8]:
def expand_contractions(reviews):
    
    """
    Routine to expand English contractions, like "isn't" --> "is not".
    This is because "isn't good" and "wasn't good" will both expand to produce the bi-gram "not good".
    The pooled phrase should have more predictive power than the original two phrases.
    """

    # Dictionary of English contractions. Taken from StackOverflow post, which borrowed it from Wikipedia:
    # https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

    contractions = { 
        "\\bain't\\b": "am not",
        "\\baren't\\b": "are not",
        "\\bcan't\\b": "cannot",
        "\\bcan't've\\b": "cannot have",
        "\\b'cause\\b": "because",
        "\\bcould've\\b": "could have",
        "\\bcouldn't\\b": "could not",
        "\\bcouldn't've\\b": "could not have",
        "\\bdidn't\\b": "did not",
        "\\bdoesn't\\b": "does not",
        "\\bdon't\\b": "do not",
        "\\bhadn't\\b": "had not",
        "\\bhadn't've\\b": "had not have",
        "\\bhasn't\\b": "has not",
        "\\bhaven't\\b": "have not",
        "\\bhe'd\\b": "he would",
        "\\bhe'd've\\b": "he would have",
        "\\bhe'll\\b": "he will",
        "\\bhe'll've\\b": "he will have",
        "\\bhe's\\b": "he is",
        "\\bhow'd\\b": "how did",
        "\\bhow'd'y\\b": "how do you",
        "\\bhow'll\\b": "how will",
        "\\bhow's\\b": "how is",
        "\\bi'd\\b": "i would",
        "\\bi'd've\\b": "i would have",
        "\\bi'll\\b": "i will",
        "\\bi'll've\\b": "i will have",
        "\\bi'm\\b": "i am",
        "\\bi've\\b": "i have",
        "\\bisn't\\b": "is not",
        "\\bit'd\\b": "it would",
        "\\bit'd've\\b": "it would have",
        "\\bit'll\\b": "it will",
        "\\bit'll've\\b": "it will have",
        "\\bit's\\b": "it is",
        "\\blet's\\b": "let us",
        "\\bma'am\\b": "madam",
        "\\bmayn't\\b": "may not",
        "\\bmight've\\b": "might have",
        "\\bmightn't\\b": "might not",
        "\\bmightn't've\\b": "might not have",
        "\\bmust've\\b": "must have",
        "\\bmustn't\\b": "must not",
        "\\bmustn't've\\b": "must not have",
        "\\bneedn't\\b": "need not",
        "\\bneedn't've\\b": "need not have",
        "\\bo'clock\\b": "of the clock",
        "\\boughtn't\\b": "ought not",
        "\\boughtn't've\\b": "ought not have",
        "\\bshan't\\b": "shall not",
        "\\bsha'n't\\b": "shall not",
        "\\bshan't've\\b": "shall not have",
        "\\bshe'd\\b": "she would",
        "\\bshe'd've\\b": "she would have",
        "\\bshe'll\\b": "she will",
        "\\bshe'll've\\b": "she will have",
        "\\bshe's\\b": "she is",
        "\\bshould've\\b": "should have",
        "\\bshouldn't\\b": "should not",
        "\\bshouldn't've\\b": "should not have",
        "\\bso've\\b": "so have",
        "\\bso's\\b": "so is",
        "\\bthat'd\\b": "that would",
        "\\bthat'd've\\b": "that would have",
        "\\bthat's\\b": "that is",
        "\\bthere'd\\b": "there would",
        "\\bthere'd've\\b": "there would have",
        "\\bthere's\\b": "there is",
        "\\bthey'd\\b": "they would",
        "\\bthey'd've\\b": "they would have",
        "\\bthey'll\\b": "they will",
        "\\bthey'll've\\b": "they will have",
        "\\bthey're\\b": "they are",
        "\\bthey've\\b": "they have",
        "\\bto've\\b": "to have",
        "\\bwasn't\\b": "was not",
        "\\bwe'd\\b": "we would",
        "\\bwe'd've\\b": "we would have",
        "\\bwe'll\\b": "we will",
        "\\bwe'll've\\b": "we will have",
        "\\bwe're\\b": "we are",
        "\\bwe've\\b": "we have",
        "\\bweren't\\b": "were not",
        "\\bwhat'll\\b": "what will",
        "\\bwhat'll've\\b": "what will have",
        "\\bwhat're\\b": "what are",
        "\\bwhat's\\b": "what is",
        "\\bwhat've\\b": "what have",
        "\\bwhen's\\b": "when is",
        "\\bwhen've\\b": "when have",
        "\\bwhere'd\\b": "where did",
        "\\bwhere's\\b": "where is",
        "\\bwhere've\\b": "where have",
        "\\bwho'll\\b": "who will",
        "\\bwho'll've\\b": "who will have",
        "\\bwho's\\b": "who is",
        "\\bwho've\\b": "who have",
        "\\bwhy's\\b": "why is",
        "\\bwhy've\\b": "why have",
        "\\bwill've\\b": "will have",
        "\\bwon't\\b": "will not",
        "\\bwon't've\\b": "will not have",
        "\\bwould've\\b": "would have",
        "\\bwouldn't\\b": "would not",
        "\\bwouldn't've\\b": "would not have",
        "\\by'all\\b": "you all",
        "\\by'all'd\\b": "you all would",
        "\\by'all'd've\\b": "you all would have",
        "\\by'all're\\b": "you all are",
        "\\by'all've\\b": "you all have",
        "\\byou'd\\b": "you would",
        "\\byou'd've\\b": "you would have",
        "\\byou'll\\b": "you will",
        "\\byou'll've\\b": "you will have",
        "\\byou're\\b": "you are",
        "\\byou've\\b": "you have"
    }
    
    # Replace all contractions in all reviews.
    for contraction in contractions:
        reviews = reviews.str.replace(contraction, contractions[contraction], regex=True)
        
    return reviews

In [9]:
# Routine to preprocess text: strip out HTML, convert to lowercase, and expand English contractions.

def preprocess_reviews(reviews):
    """
    Routine to preprocess text: strip out HTML, convert to lowercase, and expand English contractions.
    """

    # Remove HTML tags
    reviews = reviews.str.replace('<.*?>', ' ', regex=True)
    # Convert to lowercase
    reviews = reviews.str.lower()
    # Expand English contractions
    reviews = expand_contractions(reviews)
    
    return reviews

In [10]:
# Preprocess all training text
all_train["review"] = preprocess_reviews(all_train["review"])

In [11]:
# Vectorize the reviews

vectorizer = CountVectorizer(
    #preprocessor=lambda x: x.lower(), # Convert to lowercase
    stop_words=stopwords,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                     # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b"    # Use word tokenizer, but don't split on apostrophes
)

dtm_train = vectorizer.fit_transform(all_train["review"])

In [12]:
# View the number of ngrams
feature_ngrams = vectorizer.get_feature_names_out()
feature_ngrams.shape

(31701,)

In [10]:
# Output features to file
np.savetxt("all_train_features.txt", feature_ngrams, fmt="%s", delimiter=",")

### Use t-test to identify strongest 2000 positive and negative terms

In [13]:
# Try t-test to identify terms that are strongly associated with only positive or only negative reviews.

# Separate positive and negative reviews
dtm_pos = dtm_train[all_train.sentiment == 1, :]
dtm_neg = dtm_train[all_train.sentiment == 0, :]

In [14]:
# Count positive and negative reviews
dtm_pos_count = dtm_pos.shape[0]
dtm_neg_count = dtm_neg.shape[0]
dtm_pos_count, dtm_neg_count

(62385, 62615)

In [15]:
# Define variables to hold means and variances of positive & negative reviews
dtm_pos_means = np.empty(feature_ngrams.shape[0])
dtm_pos_vars = np.empty(feature_ngrams.shape[0])

dtm_neg_means = np.empty(feature_ngrams.shape[0])
dtm_neg_vars = np.empty(feature_ngrams.shape[0])

In [16]:
# Compute feature means and sample variances using dense matrices instead of sparse matrices, one column at a time.
# This is because computing variance of sparse matrices is reported to have numerical instability.
# Use one column at a time to avoid consuming too much memory.
# Approach taken from StackOverflow post:
# https://stackoverflow.com/questions/12169611/how-do-i-compute-the-variance-of-a-column-of-a-sparse-matrix-in-scipy

for col in range(feature_ngrams.shape[0]):
    pos_col_array = dtm_pos[:, col].toarray()
    dtm_pos_means[col] = np.mean(pos_col_array)
    dtm_pos_vars[col] = np.var(pos_col_array, ddof=1)
    
    neg_col_array = dtm_neg[:, col].toarray()
    dtm_neg_means[col] = np.mean(neg_col_array)
    dtm_neg_vars[col] = np.var(neg_col_array, ddof=1)

In [17]:
# For each term / ngram, compute t-statistic for two independent samples.
# Hmmm...they're not independent, but we can't really pool the variance...

t_statistics = (dtm_pos_means - dtm_neg_means) / np.sqrt((dtm_pos_vars/dtm_pos_count) + (dtm_neg_vars/dtm_neg_count))
t_statistics.shape

(31701,)

In [18]:
feature_statistic_df = pd.DataFrame({"feature": feature_ngrams.tolist(), "statistic": t_statistics.tolist()})

In [17]:
# Look at top 50 positive words
feature_statistic_df.sort_values(by="statistic", ascending=False).iloc[0:50, :]

,feature,statistic
10874,great,73.982716
8232,excellent,58.862798
31186,wonderful,53.246837
3247,best,53.155283
17806,of best,51.324386
19152,one of best,48.932390
14764,love,43.829750
20103,perfect,41.751995
1406,amazing,40.271278
2936,beautiful,39.589658


In [18]:
# Look at bottom 50 words (most negative)
feature_statistic_df.sort_values(by="statistic").iloc[0:50, :]

,feature,statistic
2537,bad,-94.755462
31315,worst,-84.764359
29641,waste,-68.997657
2465,awful,-64.850629
17113,not even,-58.121304
25264,terrible,-57.600314
31306,worse,-53.553785
3598,boring,-52.661545
24670,stupid,-50.814659
16835,no,-50.695585


In [19]:
# How many terms meet the 0.05 significance threshold?
len(feature_statistic_df[feature_statistic_df.statistic >= 1.645])

11752

In [20]:
# Check for class imbalance
all_train.groupby(["sentiment"]).count()

,review
sentiment,
0,62615
1,62385


In [21]:
# Keep the top 2000 terms by magnitude of t-statistic

n_terms = 2000

feature_statistic_df["abs_statistic"] = abs(feature_statistic_df["statistic"])

top_features = feature_statistic_df.sort_values(by="abs_statistic", ascending=False).iloc[:n_terms, 0]
top_features

2537                     bad
31315                  worst
10874                  great
29641                  waste
2465                   awful
                ...         
24033                sounded
16860              no excuse
26662    this movie horrible
24173                spot on
17214            not in good
Name: feature, Length: 2000, dtype: object

In [22]:
# Add any terms that only appeared in positive reviews or only in negative reviews.
only_positive = feature_ngrams[np.logical_and((dtm_pos_means > 0), (dtm_neg_means == 0))]
only_negative = feature_ngrams[np.logical_and((dtm_pos_means == 0), (dtm_neg_means > 0))]
only_negative

array(['avoid like', 'bad as this', 'bad music', 'could not save',
       'crap like this', 'do not waste money', 'easily worst', 'gave 2',
       'how not to make', 'instead of 1', 'manos', 'not funny not',
       'not waste money on', 'not waste time or', 'this by far worst',
       'this drivel', 'this dull', 'this junk', 'this lame',
       'this piece of garbage', 'this rubbish', 'this stinker',
       'this tripe', 'this turkey', 'this waste', 'this waste of',
       'waste time or', 'waste time or money', 'waste time with this',
       'worst movie ever made', 'worst movies ever seen'], dtype=object)

In [23]:
top_features = list(set(top_features.tolist() + only_positive.tolist() + only_negative.tolist()))
top_features

['on edge of seat',
 'perfect',
 'touch',
 'thinking',
 'did not even',
 'one of greatest',
 'film excellent',
 'rest of movie',
 'to perfection',
 'tense',
 'film works',
 'series',
 'with great',
 'worst thing',
 'dreams',
 '1',
 '8 out of',
 'be ashamed',
 'hours of',
 'surprisingly',
 'would',
 'instead of',
 'entire movie',
 'did great job',
 'sorry',
 'basically',
 'timeless',
 'of most',
 'do not',
 'painfully',
 'meaningless',
 'nothing else',
 'shows how',
 '1 10',
 'guy',
 'may',
 'films',
 'seemed to',
 'ruined',
 'in movie',
 'at times',
 'of worst',
 'thing that',
 'looks',
 'lack',
 'awful acting',
 'this garbage',
 'could not save',
 'appalling',
 'perfection',
 'to make',
 'movie no',
 'bored',
 'even remotely',
 'episodes',
 'later',
 'times',
 'impressive',
 'begins',
 'bother to',
 'only good',
 'dialog',
 'boring movie',
 'not good',
 'to make movie',
 'fails to',
 'about only',
 'movie would',
 'please do',
 'stupid movie',
 'worst ever seen',
 'man',
 'f',
 'then'

In [24]:
len(top_features)

2009

In [25]:
# Output the top features to file
np.savetxt(f"top_{len(top_features)}_features.txt", top_features, fmt="%s", delimiter=",")

### Find best ridge-regression model parameters

In [26]:
# Vectorize the full training set using the top 2000 features

top_feature_vectorizer = CountVectorizer(
    vocabulary=top_features,          # The top 200 features
    stop_words=stopwords,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                     # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b"    # Use word tokenizer, but don't split on apostrophes
)

dtm_vocab_train = top_feature_vectorizer.fit_transform(all_train["review"])

In [27]:
from sklearn.linear_model import LogisticRegressionCV

In [28]:
grid_search = LogisticRegressionCV(Cs=10, cv=5, penalty="l2", scoring="roc_auc", max_iter=100000, random_state=SEED, verbose=1)

In [29]:
all_train_y = all_train["sentiment"]

In [30]:
grid_search.fit(dtm_vocab_train, all_train_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.9min finished


LogisticRegressionCV(cv=5, max_iter=100000, random_state=4031,
                     scoring='roc_auc', verbose=1)

In [31]:
best_C = grid_search.C_[0]
best_C   # best_C is 166.81005372

166.81005372000558

In [32]:
grid_search.scores_

{1: array([[0.90935775, 0.94354946, 0.96294727, 0.97165266, 0.9748084 ,
         0.97552203, 0.97560437, 0.97560651, 0.97560641, 0.97560652],
        [0.91301104, 0.9465028 , 0.96414   , 0.97174582, 0.9744883 ,
         0.97511399, 0.97520503, 0.97521818, 0.97521824, 0.97521816],
        [0.91137743, 0.94582756, 0.96427532, 0.9722274 , 0.97494584,
         0.97547194, 0.97554306, 0.97555951, 0.97555951, 0.97555951],
        [0.91270893, 0.9457004 , 0.96344614, 0.97148508, 0.97442537,
         0.97511311, 0.97520429, 0.97520059, 0.97520056, 0.9752005 ],
        [0.91288587, 0.94584362, 0.96394406, 0.97208492, 0.97515224,
         0.97586396, 0.97595001, 0.97593079, 0.97593079, 0.97593085]])}

### How well do the 2000-ish terms predict movie review sentiment?

In [33]:
# First, preprocess training and test sets from each split. This way it's faster to try multiple model settings.

train_X_preprocessed = []
train_y_list = []

test_X_preprocessed = []
test_y_list = []

for i in range(len(train_datasets)):

    train_X_preprocessed.append(top_feature_vectorizer.fit_transform(preprocess_reviews(train_datasets[i]["review"])))
    train_y_list.append(train_datasets[i]["sentiment"])
    
    test_X_preprocessed.append(top_feature_vectorizer.transform(preprocess_reviews(test_datasets[i]["review"])))
    test_y_list.append(test_ys[i]["sentiment"])

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [36]:
pred_y = []
auc_scores = []

for i in range(len(train_X_preprocessed)):
    model = LogisticRegression(C=best_C, penalty="l2", max_iter=100000, random_state=SEED)
    
    model.fit(train_X_preprocessed[i], train_y_list[i])
    
    pred_y.append(model.predict_proba(test_X_preprocessed[i])[:, 1])  # Predict probabilities for class 1 (positive review)
    
    auc_score = roc_auc_score(test_y_list[i], pred_y[i])
    auc_scores.append(auc_score)
    
    print(f"AUC of split {i+1}: {auc_score}")

AUC of split 1: 0.956204704468669
AUC of split 2: 0.9564355005818861
AUC of split 3: 0.9546606942152958
AUC of split 4: 0.9565497321918287
AUC of split 5: 0.9548672825223585


## Reduce Vocabulary

Need to reduce vocabulary to under 100 feature for full points.

<div class="alert alert-block alert-danger">
<b>Ignore code below this line</b>
</div>

### Try Support Vector Classifier with kernel

See if the incorporation of interaction terms increases AUC.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [37]:
# Try RBF kernel

from sklearn import svm

model = svm.SVC(kernel="rbf", degree=3, probability=True, random_state=SEED)
model.fit(train_X_preprocessed[0], train_y_list[0])
pred_y = model.predict_proba(test_X_preprocessed[0])[:, 1]  # Predict probabilities for class 1 (positive review)

In [38]:
pred_y

array([7.25840412e-01, 5.61584751e-02, 9.54795041e-01, ...,
       9.99999807e-01, 1.66210730e-01, 6.08238567e-05])

In [39]:
roc_auc_score(test_y_list[0], pred_y)

0.9550786512149552

In [43]:
# Try polynomial kernel

model = svm.SVC(kernel="poly", degree=3, probability=True, random_state=SEED)
model.fit(train_X_preprocessed[0], train_y_list[0])
pred_y = model.predict_proba(test_X_preprocessed[0])[:, 1]  # Predict probabilities for class 1 (positive review)

In [44]:
roc_auc_score(test_y_list[0], pred_y)

0.9095020097725768

In [ ]:
# Try grid search over SVC regularization parameter
# *** GRID SEARCH RUNS > 18 HOURS WITHOUT COMPLETING ***

#svc_model = svm.SVC(kernel="rbf", probability=True, random_state=SEED)

#param_grid = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
#grid_search = GridSearchCV(estimator=svc_model, param_grid=param_grid, cv=5, scoring="roc_auc")
#grid_search.fit(dtm_vocab_train, all_train_y)
#best_svc_params = grid_search.best_estimator_
#best_svc_C = best_lasso_params.get_params()["C"]



### Reduce the number of terms to under 1000

## Attempt at lemmatization

In [ ]:
# Now add lemmatizer to preprocessing.
# Code taken from StackOverflow post: https://stackoverflow.com/questions/47423854/sklearn-adding-lemmatizer-to-countvectorizer

# To make this work, need to deal with punctuation. Also need to provide POS tags, like here:
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
lemma_vectorizer = CountVectorizer(
    lowercase=True,                   # Convert to lowercase
    tokenizer=LemmaTokenizer(),       # Lemmatization
    stop_words=stopwords,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                     # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b"    # Use word tokenizer
)

lemma_dtm_train = lemma_vectorizer.fit_transform(train['review'])

In [ ]:
# View some of the ngrams identified
lemma_vectorizer.get_feature_names_out()

In [ ]:
# View the number of ngrams
lemma_vectorizer.get_feature_names_out().shape

In [ ]:
# Encode as Unicode
feature_names = [l.encode("utf-8") for l in lemma_vectorizer.get_feature_names_out()]

# Output features to file
with open("split_1/lemma_vectorizer_features.txt", "wb") as f:
    for l in feature_names:
        f.write(b'%s\n'%l)

np.savetxt("split_1/lemma_vectorizer_features.txt", lemma_vectorizer.get_feature_names_out(), fmt="%s", delimiter=",")

## Try stemming words before computing their predictive power

In [ ]:
train_stem = train_datasets[3]
# Remove HTML tags
train_stem["review"] = train_stem["review"].str.replace('<.*?>', ' ', regex=True)

In [ ]:
train_stem

In [ ]:
# Remove all punctulation besides apostrophes.
# Regular expression from StackOverflow post:
# https://stackoverflow.com/questions/59877761/how-to-strip-string-from-punctuation-except-apostrophes-for-nlp
train_stem["review"] = train_stem["review"].str.replace('[^\w\d\s\']+', '', regex=True)

In [ ]:
train_stem

In [ ]:
# Try whitespace tokenizer to avoid splitting words on apostrophes
from nltk.tokenize import WhitespaceTokenizer
tk = WhitespaceTokenizer()
p_stemmer = PorterStemmer()

In [ ]:
#from nltk import word_tokenize

for i in range(len(train_stem)):
    train_stem.iloc[i, 2] = " ".join([p_stemmer.stem(review_token) for review_token in tk.tokenize(train_stem.iloc[i, 2])
                                      if review_token not in stopwords])

In [ ]:
train_stem

In [ ]:
# PorterStemmer class to add to preprocessing.
class StemmerPorter(object):
    def __init__(self):
        self.wnl = PorterStemmer()
        self.tk = WhitespaceTokenizer()
    def __call__(self, articles):
        return [self.wnl.stem(t) for t in self.tk.tokenize(articles) if t not in stopwords]

In [ ]:
stem_vectorizer = CountVectorizer(
    lowercase=True,                   # Convert to lowercase
    tokenizer=StemmerPorter(),        # Stemming
    stop_words=stopwords,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                     # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r'[^\w\d\s\']+'     # Keep apostrophes while removing other punctuation
)

dtm_train_stem = stem_vectorizer.fit_transform(train_stem['review'])

In [ ]:
# View some of the ngrams identified
stem_vectorizer.get_feature_names_out()

In [ ]:
# Encode as Unicode
stem_feature_names = [l.encode("utf-8") for l in stem_vectorizer.get_feature_names_out()]

# Output features to file
with open("split_1/stem_vectorizer_features.txt", "wb") as f:
    for l in stem_feature_names:
        f.write(b'%s\n'%l)


In [ ]:
# Try t-test to identify terms that are strongly associated with only positive or only negative reviews.

dtm_stem_array = dtm_train_stem.toarray()
dtm_stem_pos = dtm_stem_array[train_stem.sentiment == 1, :]
dtm_stem_neg = dtm_stem_array[train_stem.sentiment == 0, :]

In [ ]:
dtm_stem_pos_count = dtm_stem_pos.shape[0]
dtm_stem_neg_count = dtm_stem_neg.shape[0]
dtm_stem_pos_count, dtm_stem_neg_count

In [ ]:
dtm_stem_pos_means = np.mean(dtm_stem_pos, axis=0)
dtm_stem_pos_vars = np.var(dtm_stem_pos, axis=0, ddof=1)

In [ ]:
dtm_stem_neg_means = np.mean(dtm_stem_neg, axis=0)
dtm_stem_neg_vars = np.var(dtm_stem_neg, axis=0, ddof=1)

In [ ]:
# For each term / ngram, compute t-statistic for two independent samples.
# Hmmm...they're not independent, but we can't really pool the variance...

stem_t_statistics = (dtm_stem_pos_means - dtm_stem_neg_means) / np.sqrt((dtm_stem_pos_vars/dtm_stem_pos_count) + (dtm_stem_neg_vars/dtm_stem_neg_count))

In [ ]:
stem_feature_ngrams = stem_vectorizer.get_feature_names_out()

In [ ]:
stem_feature_statistic_df = pd.DataFrame({"feature": stem_feature_ngrams.tolist(), "statistic": stem_t_statistics.tolist()})

In [ ]:
# How many terms meet the 0.05 significance threshold?

len(feature_statistic_df[abs(feature_statistic_df.statistic) >= 1.645])

In [ ]:
# Look at top 50 positive ngrams
stem_feature_statistic_df.sort_values(by="statistic", ascending=False).iloc[0:50, :]

In [ ]:
# Look at bottom 50 ngrams (most negative)
stem_feature_statistic_df.sort_values(by="statistic").iloc[0:50, :]

In [ ]:
# Choose the 2000 most predictive tokens based on their t-statistics
n_tokens = 2000

stem_feature_statistic_df["abs_statistic"] = abs(stem_feature_statistic_df["statistic"])

In [ ]:
stem_predictive_tokens = stem_feature_statistic_df.sort_values(by="abs_statistic", ascending=False).iloc[:n_tokens, 0]

In [ ]:
stem_predictive_tokens

In [ ]:
# Add words that only appeared in positive reviews
dtm_stem_array.shape

In [ ]:
stem_feature_ngrams.shape

In [ ]:
only_positive = stem_feature_ngrams[np.logical_and((dtm_stem_pos_means > 0), (dtm_stem_neg_means == 0))]
only_negative = stem_feature_ngrams[np.logical_and((dtm_stem_pos_means == 0), (dtm_stem_neg_means > 0))]

In [ ]:
only_negative